# 04 Optimize

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')
from optimize import optimize_tourney

In [2]:
df = pd.read_csv('../data/opt_input.csv')
df.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,...,Key,Win_Prob,RPI_Strong,RPI_Weak,RPI_Strong_Dummy,RPI_Weak_Dummy,Win_Prob_glm_temp,Win_Prob_glm,Win_Prob_gbm_temp,Win_Prob_gbm
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,...,115,0.992647,2,128,1,1,0.948936,0.948936,0.968941,0.968941
1,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1347,...,115,0.007353,2,128,1,1,0.948936,0.051064,0.968941,0.031059
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,...,207,0.811594,2,61,1,1,0.796519,0.796519,0.869192,0.869192
3,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1439,...,207,0.188406,2,61,1,1,0.796519,0.203481,0.869192,0.130808
4,2018,R2W1,R1W1,R1W8,2,1347,Radford,1439,Virginia Tech,1347,...,208,0.254310,128,61,1,1,0.173994,0.826006,0.223140,0.776860


In [3]:
# weight win prob by round
df['Win_Prob_weighted'] = [w*(2**(r-1)) for w,r in zip(df['Win_Prob'], df['Round'])]
df['Win_Prob_glm_weighted'] = [w*(2**(r-1)) for w,r in zip(df['Win_Prob_glm'], df['Round'])]
df['Win_Prob_gbm_weighted'] = [w*(2**(r-1)) for w,r in zip(df['Win_Prob_gbm'], df['Round'])]

df.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,...,RPI_Weak,RPI_Strong_Dummy,RPI_Weak_Dummy,Win_Prob_glm_temp,Win_Prob_glm,Win_Prob_gbm_temp,Win_Prob_gbm,Win_Prob_weighted,Win_Prob_glm_weighted,Win_Prob_gbm_weighted
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,...,128,1,1,0.948936,0.948936,0.968941,0.968941,0.992647,0.948936,0.968941
1,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1347,...,128,1,1,0.948936,0.051064,0.968941,0.031059,0.007353,0.051064,0.031059
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,...,61,1,1,0.796519,0.796519,0.869192,0.869192,1.623188,1.593037,1.738384
3,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1439,...,61,1,1,0.796519,0.203481,0.869192,0.130808,0.376812,0.406963,0.261616
4,2018,R2W1,R1W1,R1W8,2,1347,Radford,1439,Virginia Tech,1347,...,61,1,1,0.173994,0.826006,0.223140,0.776860,0.508621,1.652011,1.553721


In [4]:
opt_results = optimize_tourney(df, 'Win_Prob_gbm', 'max')

In [5]:
opt_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,...,1395_6,1138_6,1397_6,1400_6,1401_6,1274_6,1403_6,1276_6,1277_6,Chosen
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,...,0,0,0,0,0,0,0,0,0,1.0
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,...,0,0,0,0,0,0,0,0,0,1.0
8,2018,R1W8,W08,W09,1,1439,Virginia Tech,1104,Alabama,1439,...,0,0,0,0,0,0,0,0,0,1.0
12,2018,R3W1,R2W1,R2W4,3,1437,Villanova,1455,Wichita St,1437,...,0,0,0,0,0,0,0,0,0,1.0
22,2018,R1W4,W04,W13,1,1455,Wichita St,1267,Marshall,1455,...,0,0,0,0,0,0,0,0,0,1.0


In [6]:
opt_results.to_csv('../data/opt_results.csv', index=False)

In [7]:
df.to_csv('../data/opt_input_v2.csv', index=False)

### Simulate by picking team with highest prob at each round

In [13]:
win_var = 'Win_Prob_glm'
winners = list(df['TeamID_Strong']) + list(df['TeamID_Weak'])
output = pd.DataFrame()

for r in [1,2,3,4,5,6]:
    # filter to round
    rnd = df.loc[df['Round']==r]
    # drop duplicates
    rnd = rnd.drop_duplicates(['Season', 'Slot', 'StrongSeed', 'WeakSeed', 'Round', 'TeamID_Strong', 'TeamID_Weak'])
    # limit down to potential teams
    rnd = rnd.loc[rnd['TeamID_Strong'].isin(winners)]
    rnd = rnd.loc[rnd['TeamID_Weak'].isin(winners)]
    # pick winner
    rnd['TeamID_Winner'] = [s if p>=0.5 else w for s,w,p in zip(rnd['TeamID_Strong'], rnd['TeamID_Weak'], rnd[win_var])]
    winners = list(rnd['TeamID_Winner'].values)
    output = output.append(rnd)

output.to_csv('../data/opt_results_simple.csv', index=False)    
